In [1]:
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob 
import cv2
import os
from PIL import Image
import shutil
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
import pandas as pd
import glob 
from PIL import Image
import re
import face_detection
from concurrent.futures import ThreadPoolExecutor

/home/obasit/50024/kaggle/kaggle_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-04 21:04:12.195607: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 21:04:13.026638: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [7]:
mtcnn = MTCNN(
    image_size=160, margin=40, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device, select_largest=False,
    selection_method='largest_over_threshold', keep_all=True
)

In [8]:
output_classes_df = pd.read_csv('/home/obasit/50024/kaggle/data/train/train.csv', index_col=0)


In [ ]:
# for celeb_og in output_classes_df['Category'].unique().tolist():
#     celeb = celeb_og.replace(' ', '_')
#     # os.mkdir('/home/obasit/50024/kaggle/data/train/train_mtcnn/' + celeb)
#     print(celeb)
#     for img in output_classes_df[output_classes_df["Category"] == celeb_og]['File Name'].to_list():
#         source_file = '/home/obasit/50024/kaggle/data/train/train/' + img
#         y = '/home/obasit/50024/kaggle/data/train/train_mtcnn/' + img

#         x = Image.open(source_file)
#         if x.mode != 'RGB':
#             x = x.convert('RGB')
#         # print(y)
#         mtcnn(x, save_path=y)
#         # shutil.copy(source_file, destination_file)

In [3]:
for celeb_og in output_classes_df['Category'].unique().tolist():
    celeb = celeb_og.replace(' ', '_')
    os.mkdir('/home/obasit/50024/kaggle/data/train/train_mtcnn/' + celeb)
    print(celeb)
    for img in output_classes_df[output_classes_df["Category"] == celeb_og]['File Name'].to_list():
        source_file = '/home/obasit/50024/kaggle/data/train/train/' + img
        y = '/home/obasit/50024/kaggle/data/train/train_mtcnn/' + celeb + '/' + img

        # x = Image.open(source_file)

        # print(y)
        shutil.copy(source_file, y)

Audrey_Tautou
Adam_Sandler
Anna_Paquin
Ava_Gardner
Amy_Adams
Angelina_Jolie
Bill_Paxton
Alan_Alda
Abigail_Breslin
Alicia_Vikander
Albert_Finney
Aaron_Judge
Beyonce_Knowles
Andrew_Lincoln
Armin_Mueller-Stahl
Andy_Murray
Amy_Ryan
Alan_Arkin
Adrien_Brody
Analeigh_Tipton
Amanda_Bynes
Anne_Bancroft
Albert_Brooks
Barbra_Streisand
Amanda_Peet
bella_thorne
Andy_Samberg
Annette_Bening
Anthony_Perkins
Alice_Eve
Berenice_Bejo
Amanda_Crew
AnnaSophia_Robb
Billy_Bob_Thornton
Aaron_Paul
Alan_Rickman
Bella_Hadid
Arnold_Schwarzenegger
Anjelica_Huston
Anne_Baxter
Betty_White
Adele
Art_Carney
Adriana_Barraza
Alex_Pettyfer
Barabara_Palvin
Alessandra_Ambrosio
Andreea_Diaconu
Al_Roker
Benedict_Cumberbatch
Ben_Kingsley
Anne_Hathaway
Anna_Friel
Alx_James
Andrew_Garfield
Antonio_Banderas
Anna_Kendrick
Amanda_Seyfried
Anthony_Mackie
Ali_Larter
Bill_Daley
Ben_Johnson
Amber_Heard
Anderson_Cooper
Alexander_Skarsgard
Barry_Pepper
Bernie_Mac
Alec_Baldwin
Adriana_Lima
Aaron_Taylor-Johnson
Ashley_Judd
Bette_Midler
Ash

In [9]:
def filter_RGB(image):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return image

class FilterRGB(object):
    def __call__(self, img):
        filtered_img = filter_RGB(img)
        if filtered_img is None:
            return None
        return filtered_img
    
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    FilterRGB(),
])

# Path to your folder containing the images
folder_path = "/home/obasit/50024/kaggle/data/test/test/"
new_folder_path = "/export2/obasit/50024/kaggle_data/test/test_mtcnn_inlcuding_grey_512_20_smallest_face_prob_98/"

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter only .jpg files
jpg_files = [file for file in file_list if file.endswith(".jpg")]

# Iterate through each file
for file_name in jpg_files:
    img_path = folder_path + file_name
    image = Image.open(img_path)

    image = transform(image)

    try:
        mtcnn(image, save_path=new_folder_path + file_name)
    except:
        print(file_name)

/home/obasit/50024/kaggle/kaggle_env/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [13]:
import face_detection

# Initialize detector
detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3, device='cpu')
# Read image
# img = cv2.imread("/home/obasit/50024/kaggle/data/test/test/1848.jpg")
pil_image = Image.open("/home/obasit/50024/kaggle/data/test/test/1848.jpg")
pil_image = transform(pil_image)
img = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
# Getting detections
faces = detector.detect(img)

for (xmin, ymin, xmax, ymax, d) in faces:
    roi = img[int(ymin):int(ymax), int(xmin):int(xmax)]

plt.imshow(roi)

/home/obasit/50024/kaggle/kaggle_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/obasit/50024/kaggle/kaggle_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


NameError: name 'transform' is not defined

In [7]:
def filter_RGB(image):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return image

class FilterRGB(object):
    def __call__(self, img):
        filtered_img = filter_RGB(img)
        if filtered_img is None:
            return None
        return filtered_img
    
transform = transforms.Compose([
    transforms.Resize((1024, 1024)),
    FilterRGB(),
])

post_transform = transforms.Compose([
    transforms.Resize((160, 160)),
])

detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

# Path to your folder containing the images
folder_path = "/home/obasit/50024/kaggle/data/test/test/"
new_folder_path = "/export2/obasit/50024/kaggle_data/test/test_dfsd_inlcuding_grey/"

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter only .jpg files
jpg_files = [file for file in file_list if file.endswith(".jpg")]

# Iterate through each file
for file_name in jpg_files:
    
    numbers = re.findall(r'\d+', file_name)
    number = ''.join(numbers)
    img_path = folder_path + file_name
    # img_path = folder_path + '1948.jpg'
    pil_image = Image.open(img_path)
    pil_image = transform(pil_image)

    img = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    # Getting detections
    faces = detector.detect(img)

    # print(img_path)

    count = 0
    if len(faces > 0):
        for (xmin, ymin, xmax, ymax, d) in faces:
            xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
            if(xmin < 0):
                xmin = 0
            if(ymin < 0):
                ymin = 0
            if(xmax > img.shape[1]):
                xmax = img.shape[1]
            if(ymax > img.shape[0]):
                ymax = img.shape[0]

            if (xmax - xmin < 40 | ymax - ymin < 40):
                continue

            # print(xmin, ymin, xmax, ymax)
            # print(img.shape)

            roi = img[ymin:ymax, xmin:xmax]
            roi = cv2.resize(roi, (160, 160))
            if (count == 0):
                cv2.imwrite(new_folder_path + number + '.jpg', roi)
            else:
                cv2.imwrite(new_folder_path + number + '_' + str(count) + '.jpg', roi)
            count = count+1
    else:
        print(file_name)


2777.jpg
3558.jpg
4477.jpg
825.jpg
4757.jpg
282.jpg
3148.jpg
317.jpg
1997.jpg
210.jpg
927.jpg
545.jpg
328.jpg
3209.jpg
1480.jpg
585.jpg
3752.jpg
1590.jpg
4610.jpg
1826.jpg
2523.jpg
609.jpg
3437.jpg
4296.jpg
1868.jpg
4888.jpg
2324.jpg
4337.jpg
2411.jpg
1336.jpg
1749.jpg
343.jpg
4237.jpg
3905.jpg
2431.jpg
4390.jpg
1499.jpg
3010.jpg
1922.jpg
3731.jpg
1658.jpg
701.jpg
3819.jpg
3568.jpg


In [48]:

# Define the filter_RGB function and FilterRGB class
def filter_RGB(image):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return image

class FilterRGB(object):
    def __call__(self, img):
        filtered_img = filter_RGB(img)
        return filtered_img

# Define the transformation pipeline
transform = transforms.Compose([
    FilterRGB(),
])

# Define the face detector (assuming this is already defined)
detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3, device=device)

def process_image(file_path):
    numbers = re.findall(r'\d+', file_path)
    number = ''.join(numbers)
    img_path = folder_path + file_path
    pil_image = Image.open(img_path)
    pil_image = transform(pil_image)
    img = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    faces = detector.detect(img)
    count = 0
    if len(faces) > 0:
        for (xmin, ymin, xmax, ymax, d) in faces:
            xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
            if(xmin < 0):
                xmin = 0
            if(ymin < 0):
                ymin = 0
            if(xmax > img.shape[1]):
                xmax = img.shape[1]
            if(ymax > img.shape[0]):
                ymax = img.shape[0]
            if (xmax - xmin < 40 | ymax - ymin < 40):
                continue
            roi = img[ymin:ymax, xmin:xmax]
            if (count == 0):
                cv2.imwrite(new_folder_path + number + '.jpg', roi)
            else:
                cv2.imwrite(new_folder_path + number + '_' + str(count) + '.jpg', roi)
            count = count+1
    else:
        print(file_path)

# Path to your folder containing the images
folder_path = "/home/obasit/50024/kaggle/data/test/test/"
new_folder_path = "/export2/obasit/50024/kaggle_data/test/test_dfsd_inlcuding_grey/"

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter only .jpg files
jpg_files = [file for file in file_list if file.endswith(".jpg")]

# Define the number of threads to use
num_threads = 8  # Adjust as needed

# Process images using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    executor.map(process_image, jpg_files)


2777.jpg
3558.jpg
4477.jpg
2583.jpg
2940.jpg
4757.jpg
3322.jpg
3148.jpg
317.jpg
2558.jpg
1997.jpg
210.jpg
927.jpg
545.jpg


/home/obasit/50024/kaggle/kaggle_env/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1480.jpg
585.jpg
3752.jpg
4610.jpg
166.jpg
2008.jpg
4923.jpg
3973.jpg
1826.jpg
2523.jpg
609.jpg
3599.jpg
1868.jpg
4888.jpg
133.jpg
2547.jpg
4337.jpg
2411.jpg
343.jpg
4237.jpg
3905.jpg
2431.jpg
4310.jpg
4390.jpg
4395.jpg
4780.jpg
1499.jpg
89.jpg
3010.jpg
1922.jpg
3731.jpg
1658.jpg
701.jpg
3819.jpg
3568.jpg


In [ ]:
from PIL import Image
import os

# Path to your folder containing the images
folder_path = "/home/obasit/50024/kaggle/data/test/test_mtcnn_cropped/"
new_folder_path = "/home/obasit/50024/kaggle/data/test/test_mtcnn_merged/"

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter only .jpg files
jpg_files = [file for file in file_list if file.endswith(".jpg")]

# Create a dictionary to store images with the same [number1]
images_dict = {}

# Iterate through each file
for file_name in jpg_files:
    parts = file_name.split('_')
    number1 = parts[0].split('.')[0]  # Extract number1 from the filename
    if number1 not in images_dict:
        images_dict[number1] = []  # Initialize an empty list for this number1
    image = Image.open(os.path.join(folder_path, file_name))  # Open the image
    images_dict[number1].append(image)  # Append the image to the list for this number1

# Concatenate images with the same number1
for number1, images_list in images_dict.items():
    # Concatenate horizontally
    concatenated_image = Image.new('RGB', (sum(img.width for img in images_list), max(img.height for img in images_list)))
    x_offset = 0
    for img in images_list:
        concatenated_image.paste(img, (x_offset, 0))
        x_offset += img.width
    # Save the concatenated image
    concatenated_image.save(os.path.join(new_folder_path, f"{number1}.jpg"))

print("Concatenation completed.")


In [9]:
import os
import threading
import numpy as np
from PIL import Image

class RGBProcessor(threading.Thread):
    def __init__(self, file_path, result_queue):
        super(RGBProcessor, self).__init__()
        self.file_path = file_path
        self.result_queue = result_queue

    def run(self):
        if self.file_path.lower().endswith('.jpg'):
            try:
                img = Image.open(self.file_path)
                img_data = np.array(img)
                
                if len(img_data.shape) == 3:
                    # RGB image
                    if img_data.shape[2] == 3:
                        mean_rgb = np.mean(img_data, axis=(0, 1))
                        std_rgb = np.std(img_data, axis=(0, 1))
                        self.result_queue.put((mean_rgb, std_rgb))
                    elif img_data.shape[2] == 4:
                        # RGBA image, ignoring alpha channel
                        mean_rgb = np.mean(img_data[:, :, :3], axis=(0, 1))
                        std_rgb = np.std(img_data[:, :, :3], axis=(0, 1))
                        self.result_queue.put((mean_rgb, std_rgb))
                    else:
                        print(f"Ignoring unsupported image format: {self.file_path}")
                elif len(img_data.shape) == 2:
                    # Grayscale image
                    mean_gray = np.mean(img_data)
                    std_gray = np.std(img_data)
                    self.result_queue.put((mean_gray, std_gray))
                else:
                    print(f"Ignoring unsupported image format: {self.file_path}")
            except Exception as e:
                print(f"Error processing {self.file_path}: {e}")

def process_folders(root_folders):
    result_queue = queue.Queue()
    threads = []
    
    for root_folder in root_folders:
        # Traverse folders
        for root, dirs, files in os.walk(root_folder):
            for file in files:
                file_path = os.path.join(root, file)
                thread = RGBProcessor(file_path, result_queue)
                threads.append(thread)
                thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    # Aggregate results
    mean_values = []
    std_values = []
    while not result_queue.empty():
        mean_val, std_val = result_queue.get()
        mean_values.append(mean_val)
        std_values.append(std_val)

    if mean_values and std_values:
        mean_values = np.array([val for val in mean_values if isinstance(val, np.ndarray)])
        std_values = np.array([val for val in std_values if isinstance(val, np.ndarray)])
        
        if len(mean_values.shape) == 2:
            overall_mean = np.mean(mean_values, axis=0)
            overall_std = np.mean(std_values, axis=0)
        else:
            overall_mean = np.mean(mean_values)
            overall_std = np.mean(std_values)
        
        print("Overall Mean:", overall_mean)
        print("Overall Std:", overall_std)

import queue

# Provide the root folders where JPG files are located
root_folders = ["/home/obasit/50024/kaggle/data/test/test", "/home/obasit/50024/kaggle/data/train/train"]
process_folders(root_folders)




Ignoring unsupported image format: /home/obasit/50024/kaggle/data/train/train/11367.jpg
Ignoring unsupported image format: /home/obasit/50024/kaggle/data/train/train/5982.jpg
Ignoring unsupported image format: /home/obasit/50024/kaggle/data/train/train/43954.jpg
Ignoring unsupported image format: /home/obasit/50024/kaggle/data/train/train/30522.jpg
Ignoring unsupported image format: /home/obasit/50024/kaggle/data/train/train/23850.jpg
Ignoring unsupported image format: /home/obasit/50024/kaggle/data/train/train/27705.jpg
Overall Mean: [121.76784405 107.24921809 100.89841174]
Overall Std: [69.23876954 64.68627036 63.39736447]


In [8]:
def find_rgba_image(folder_path):
    # Iterate over all files in the folder
    for filename in sorted(os.listdir(folder_path)):
        filepath = os.path.join(folder_path, filename)
        try:
            # Open the image
            img = Image.open(filepath)
            
            # Check if the image has RGBA mode
            if img.mode == 'RGBA':
                print(f"First RGBA image found at: {filepath}")
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
    return None, None

# Example usage
folder_path = "/home/obasit/50024/kaggle/data/train/train_mtcnn_concat"
find_rgba_image(folder_path)


(None, None)

In [5]:
import os

dir2 = "/home/obasit/50024/kaggle/data/export2_data/kaggle_data/train/train_dfsd_inlcuding_grey"
dir1 = "/home/obasit/50024/kaggle/data/export2_data/kaggle_data/train/train_mtcnn"

# Check if both directories exist
if not (os.path.exists(dir1) and os.path.exists(dir2)):
    print("One or both directories do not exist.")
    exit(1)

# Get list of folders in dir1
folders_in_dir1 = [name for name in os.listdir(dir1) if os.path.isdir(os.path.join(dir1, name))]
print(len(folders_in_dir1))
# Get list of folders in dir2
folders_in_dir2 = [name for name in os.listdir(dir2) if os.path.isdir(os.path.join(dir2, name))]

# Check if all folders in dir1 are present in dir2
missing_folders = [folder for folder in folders_in_dir1 if folder not in folders_in_dir2]

if missing_folders:
    print("Folders missing in the second directory:")
    for folder in missing_folders:
        print(folder)

100
Folders missing in the second directory:
Ava_Gardner
Beyonce_Knowles
Benjamin_Bratt
Anne_Bancroft
Bill_O_Reilly
Amy_Ryan
Aaron_Paul
Ashley_Graham
Bill_Hader
Billy_Bob_Thornton


In [7]:
import os
from collections import defaultdict
import re

def delete_repeated_files(directory):
    # Dictionary to store count of number1 occurrences
    count_dict = defaultdict(int)

    # Regex pattern to extract number1 from filename
    pattern = re.compile(r'^(\d+)_')

    # Iterate through files in directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for filename in files:
            # Check if file is a jpg
            if filename.endswith(".jpg"):
                # Extract number1 from filename
                match = pattern.match(filename)
                if match:
                    number1 = match.group(1)
                    count_dict[number1] += 1

    # Delete files with repeated number1 more than 2 times
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".jpg"):
                # Extract number1 from filename
                match = pattern.match(filename)
                if match:
                    number1 = match.group(1)
                    if count_dict[number1] > 2:
                        # os.remove(os.path.join(root, filename))
                        print(f"Deleted: {filename}")

# Provide the directory containing the jpg files
directory_path = "/home/obasit/50024/kaggle/data/export2_data/kaggle_data/train/train_dfsd_inlcuding_grey"
delete_repeated_files(directory_path)


Deleted: 44168_6.jpg
Deleted: 63473_3.jpg
Deleted: 60058_5.jpg
Deleted: 60058_4.jpg
Deleted: 5162_3.jpg
Deleted: 60058_1.jpg
Deleted: 44168_3.jpg
Deleted: 5162_4.jpg
Deleted: 21601_5.jpg
Deleted: 33125_2.jpg
Deleted: 21601_3.jpg
Deleted: 5737_4.jpg
Deleted: 11632_1.jpg
Deleted: 33125_1.jpg
Deleted: 44168_5.jpg
Deleted: 63473_1.jpg
Deleted: 60058_3.jpg
Deleted: 5737_2.jpg
Deleted: 11632_2.jpg
Deleted: 5737_3.jpg
Deleted: 44168_4.jpg
Deleted: 12132_1.jpg
Deleted: 33125_3.jpg
Deleted: 58789_1.jpg
Deleted: 5737_1.jpg
Deleted: 33125_4.jpg
Deleted: 21601_6.jpg
Deleted: 2368_3.jpg
Deleted: 2368_4.jpg
Deleted: 44168_2.jpg
Deleted: 44168_1.jpg
Deleted: 21601_1.jpg
Deleted: 60058_6.jpg
Deleted: 60058_2.jpg
Deleted: 21601_4.jpg
Deleted: 21601_2.jpg
Deleted: 63473_2.jpg
Deleted: 5162_1.jpg
Deleted: 60058_7.jpg
Deleted: 58789_3.jpg
Deleted: 11632_3.jpg
Deleted: 44168_7.jpg
Deleted: 5162_2.jpg
Deleted: 2368_1.jpg
Deleted: 12132_2.jpg
Deleted: 12132_3.jpg
Deleted: 44168_8.jpg
Deleted: 58789_2.jpg
Del

In [10]:
import os
from PIL import Image

def resize_images_in_folder(input_folder, output_folder, target_size=(160, 160)):
    """
    Resize all .jpg images in a folder recursively to the specified target size.
    Save resized images to a different folder while preserving the folder structure.

    Args:
    - input_folder (str): Path to the folder containing images.
    - output_folder (str): Path to the folder to save resized images.
    - target_size (tuple): Target size for resizing images, default is (160, 160).
    """
    for root, dirs, files in os.walk(input_folder):
        relative_path = os.path.relpath(root, input_folder)
        output_subfolder = os.path.join(output_folder, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)

        for file in files:
            if file.lower().endswith('.jpg'):
                file_path = os.path.join(root, file)
                output_path = os.path.join(output_subfolder, file)
                try:
                    # Open image
                    with Image.open(file_path) as img:
                        # Resize image
                        resized_img = img.resize(target_size)
                        # Save resized image to output folder
                        resized_img.save(output_path)
                        print(f"Resized: {output_path}")
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Example usage
if __name__ == "__main__":
    input_folder = "/home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey"
    output_folder = "/home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160"
    resize_images_in_folder(input_folder, output_folder)


Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./3262.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./1889.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./2644_9.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./3863.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./1637_1.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./1898.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./156.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./2366.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/

Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./3544.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./1061.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./3512_1.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./4650.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./2247_5.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./123_1.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./4413.jpg
Resized: /home/obasit/50024/kaggle/data/export2_data/kaggle_data/test/test_dfsd_inlcuding_grey_cropped_160/./1635_2.jpg
Resized: /home/obasit/50024/kaggle/data/export2_d